## Using lib.distances to identify hydrogen bonds

This notebook walks through how to use various functions in MDAnalysis.lib.distances to identify hydrogen bonding between certain residues and the water solvent.
Hydrogen bonds are here defined by a geometric criteria, a hydrogen-acceptor distance of 3.0A and an acceptor-hydrogen-donor angle of greater than 120 degrees.

We start with the usual imports and loading our system (here TPR and TRR).

In [74]:
import MDAnalysis as mda
from MDAnalysisTests.datafiles import TPR, TRR

import numpy as np

In [75]:
u = mda.Universe(TPR, TRR)

We then select our groups of interest.
Firstly we grab all the oxygen atoms on either ASP or GLU residues.

In [20]:
sidegroups = u.select_atoms('resname ASP GLU')

acceptors = sidegroups.select_atoms('element O')

Then we select the hydrogen atoms from the SOL (solvent) molecules.

In [22]:
solvent = u.select_atoms('resname SOL')

hydrogens = solvent.select_atoms('element H')

## Distance criteria

We first want to identify hydrogens and acceptors that are within our distance criteria of 3.0 angstrom.
A naive approach is to calculate a distance array between all acceptors and all hydrogens.:

In [76]:
%%time

da = mda.lib.distances.distance_array(acceptors.positions, hydrogens.positions, box=u.dimensions)

CPU times: user 135 ms, sys: 7.98 ms, total: 143 ms
Wall time: 143 ms


In [77]:
acc_idx, hyd_idx = np.where(da < 3.0)

## Using capped distance

This is an example of where we're not interested in all distances, but instead only those up to a given cutoff.  Using `capped_distance` is much quicker here.

In [68]:
%%time 

idx, dists = mda.lib.distances.capped_distance(acceptors.positions, hydrogens.positions, max_cutoff=3.0,
                                            box=u.dimensions)

CPU times: user 1.6 ms, sys: 3.7 ms, total: 5.3 ms
Wall time: 4.22 ms


In [51]:
acc_idx, hyd_idx = idx.T

In [69]:
# select potential hydrogen bonds to check angles
hbond_acceptors = acceptors[acc_idx]
hbond_hydrogens = hydrogens[hyd_idx]
# this is slightly complex
# we need the donor atom for each hydrogen
# we can get this by grabbing the first (and only) bonded atom for each hydrogen atom 
hbond_donors = sum(h.bonded_atoms[0] for h in hbond_hydrogens)

## Checking the angle criteria

In [71]:
angles = np.rad2deg(
    mda.lib.distances.calc_angles(hbond_acceptors.positions, hbond_hydrogens.positions, hbond_donors.positions, box=u.dimensions)
)

In [73]:
np.where(angles >= 120.0)

(array([  0,   1,   3,   4,   5,   6,   8,   9,  11,  12,  14,  15,  16,
         17,  19,  20,  21,  24,  25,  28,  29,  30,  32,  33,  36,  37,
         38,  39,  40,  41,  43,  44,  46,  48,  49,  50,  51,  53,  54,
         55,  57,  58,  60,  62,  63,  64,  66,  67,  68,  69,  70,  71,
         72,  73,  74,  75,  76,  77,  78,  80,  81,  82,  83,  86,  88,
         90,  91,  92,  93,  94,  95,  96,  98, 100, 101, 103, 104, 105,
        106, 108, 109, 110, 111, 112, 113, 115, 117, 118, 120, 121, 123,
        124, 125, 126, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 150, 153, 154, 155,
        157, 158, 160, 161, 163, 164, 166, 167, 168, 169, 172, 173, 174,
        175, 176, 178, 180, 181, 183, 184, 186, 188, 189, 190, 191, 192,
        193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206,
        209, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,
        223, 224, 225, 227, 228, 229, 230, 232, 233